In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import anndata2ri
import gdown
import scipy
import scipy.io
from rpy2.robjects import r

anndata2ri.activate()

In [2]:
%load_ext rpy2.ipython

C:\Users\Chayan\anaconda3\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
%%R
suppressPackageStartupMessages({
    library(SingleCellExperiment)
    library(Seurat)
    library(SeuratDisk)
    library(dplyr)
})

## Importing the data

In [4]:
dir = 'E:/UMINTv2/Data/kotliarov2020/'

In [5]:
adata = sc.read(dir+'expressions_hvg.h5ad')
adata

AnnData object with n_obs × n_vars = 52117 × 3999
    obs: 'batch', 'cluster_level2', 'cluster_level3', 'sample', 'cell_type', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'batch_colors', 'cell_type_colors', 'cluster_level2_colors', 'cluster_level3_colors', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'count'
    obsp: 'connectivities', 'distances'

In [6]:
# Seurat categories have to be strings
adata.obs.batch = adata.obs.batch.cat.rename_categories(["batch1", "batch2"])

In [7]:
adata_ = ad.AnnData(adata.layers['count'])
adata_.obs_names = adata.obs_names
adata_.var_names = adata.var_names
adata_.obs['cell_type'] = adata.obs['cell_type']
adata_.obs['batch'] = adata.obs['batch']

In [8]:
%%R -i adata_
rna = as.Seurat(adata_, counts='X', data=NULL)
rna

An object of class Seurat 
3999 features across 52117 samples within 1 assay 
Active assay: originalexp (3999 features, 0 variable features)


In [9]:
adata = sc.read(dir+'protein.h5ad')

In [10]:
# Seurat categories have to be strings
adata.obs.batch = adata.obs.batch.cat.rename_categories(["batch1", "batch2"])

In [11]:
adata_ = ad.AnnData(adata.layers['count'])
adata_.obs_names = adata.obs_names
adata_.var_names = adata.var_names
adata_.obs['cell_type'] = adata.obs['cell_type']
adata_.obs['batch'] = adata.obs['batch']

In [12]:
%%R -i adata_
cite = as.Seurat(adata_, counts='X', data=NULL)
cite

R[write to console]: Warning:
R[write to console]:  Feature names cannot have underscores ('_'), replacing with dashes ('-')



An object of class Seurat 
87 features across 52117 samples within 1 assay 
Active assay: originalexp (87 features, 0 variable features)


# Preproccessing

In [13]:
%%R
rna <- RenameAssays(rna, originalexp = 'RNA')
DefaultAssay(rna) <- 'RNA'
rna <- NormalizeData(rna) %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA()

R[write to console]: Renaming default assay from originalexp to RNA

R[write to console]: Performing log-normalization

R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to conso

In [14]:
%%R
cite <- RenameAssays(cite, originalexp='ADT')
DefaultAssay(cite) <- 'ADT'
VariableFeatures(cite) <- rownames(cite[["ADT"]])
cite <- NormalizeData(cite, normalization.method = 'CLR', margin = 2) %>% 
  ScaleData() %>% RunPCA(reduction.name = 'apca')


R[write to console]: Renaming default assay from originalexp to ADT

R[write to console]: Normalizing across cells



  |                                                  | 0 % ~calculating   |+                                                 | 0 % ~00s           |+                                                 | 0 % ~08m 41s       |+                                                 | 0 % ~05m 47s       |+                                                 | 0 % ~04m 21s       |+                                                 | 0 % ~05m 13s       |+                                                 | 0 % ~04m 21s       |+                                                 | 0 % ~03m 43s       |+                                                 | 0 % ~05m 26s       |+                                                 | 0 % ~04m 49s       |+                                                 | 0 % ~04m 21s       |+                                                 | 0 % ~04m 44s       |+                                                 | 0 % ~04m 21s       |+                                                 | 0 % ~04m 

  |+                                                 | 0 % ~04m 02s       |+                                                 | 0 % ~03m 60s       |+                                                 | 0 % ~03m 58s       |+                                                 | 0 % ~04m 04s       |+                                                 | 0 % ~04m 02s       |+                                                 | 0 % ~04m 00s       |+                                                 | 0 % ~03m 58s       |+                                                 | 0 % ~04m 01s       |+                                                 | 0 % ~03m 59s       |+                                                 | 0 % ~04m 05s       |+                                                 | 0 % ~04m 03s       |+                                                 | 0 % ~04m 01s       |+                                                 | 0 % ~04m 08s       |+                                                 | 0 % ~04m 

  |+                                                 | 0 % ~04m 01s       |+                                                 | 0 % ~04m 00s       |+                                                 | 0 % ~03m 59s       |+                                                 | 0 % ~04m 03s       |+                                                 | 0 % ~04m 02s       |+                                                 | 0 % ~04m 05s       |+                                                 | 0 % ~04m 11s       |+                                                 | 0 % ~04m 09s       |+                                                 | 0 % ~04m 11s       |+                                                 | 0 % ~04m 10s       |+                                                 | 0 % ~04m 08s       |+                                                 | 0 % ~04m 07s       |+                                                 | 0 % ~04m 11s       |+                                                 | 0 % ~04m 

  |+                                                 | 1 % ~04m 16s       |+                                                 | 1 % ~04m 15s       |+                                                 | 1 % ~04m 17s       |+                                                 | 1 % ~04m 17s       |+                                                 | 1 % ~04m 16s       |+                                                 | 1 % ~04m 17s       |+                                                 | 1 % ~04m 16s       |+                                                 | 1 % ~04m 15s       |+                                                 | 1 % ~04m 14s       |+                                                 | 1 % ~04m 17s       |+                                                 | 1 % ~04m 16s       |+                                                 | 1 % ~04m 15s       |+                                                 | 1 % ~04m 16s       |+                                                 | 1 % ~04m 

...

  |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s           |+++++++++++++++++++++++++++++++++++++++++++++++++ | 98% ~05s 

  |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~05s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~05s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s 

  |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 98% ~04s 

  |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 99% ~03s 

R[write to console]: PC_ 1 
Positive:  CD32-PROT, CD39-PROT, HLA-DR-PROT, CD1d-PROT, CD11c-PROT, CD11b-PROT, CD86-PROT, CD33-PROT, CD64-PROT, CD196-PROT 
	   CD31-PROT, CD40-PROT, CD141-PROT, CD13-PROT, CD163-PROT, CD123-PROT, CD69-PROT, CD20-PROT, CD19-PROT, CD1c-PROT 
	   CD71-PROT, IgD-PROT, CD366-PROT, CD21-PROT, CD185-PROT, CD34-PROT, CD38-PROT, HLA-ABC-PROT, CD24-PROT, BTLA-PROT 
Negative:  CD2-PROT, CD3-PROT, CD5-PROT, CD7-PROT, CD28-PROT, CD27-PROT, CD127-PROT, CD278 -PROT, CD4-PROT, CD279-PROT 
	   CD62L-PROT, CD45RO-PROT, CD8-PROT, KLRG1-PROT, CD314 -PROT, CD197-PROT, CD25-PROT, CD134-PROT, CD161-PROT, CD183-PROT 
	   CD194-PROT, TCRgd-PROT, CD195-PROT, CD357-PROT, CD57-PROT, CD273-PROT, MouseIgG2akappaisotype-PROT, CD117-PROT, CD223-PROT, CD138-PROT 
PC_ 2 
Positive:  CD18-PROT, CD11c-PROT, CD11b-PROT, CD64-PROT, CD33-PROT, CD244-PROT, CD86-PROT, CD13-PROT, CD14-PROT, CD163-PROT 
	   CD141-PROT, CD69-PROT, CD31-PROT, CD34-PROT, CD45RO-PROT, CX3CR1-PROT, CD1d-PROT, CD366-PROT

# Extracting scaled data

In [15]:
%%R
#data preparation for UMINT
rna_data <- as.matrix(GetAssayData(rna, slot = "scale.data", assay="RNA"))
write.csv(rna_data, 'E:/UMINTv2/Data/kotliarov50k/kotliarov50k_rna_scaled.csv')

adt_data <- as.matrix(GetAssayData(cite, slot = "scale.data", assay="ADT"))
write.csv(adt_data, 'E:/UMINTv2/Data/kotliarov50k/kotliarov50k_adt_scaled.csv')